In [ ]:

#http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data


In [32]:
import pandas as pd
import numpy as np
import time

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [2]:
#Step1: Import data
url="http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
cc_apps=pd.read_csv(url, header=None)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [3]:
#Step2-1:EDA1
cc_apps.describe()

,2,7,10,14
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [7]:
#Step2-1:EDA2 How Many Categor
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [9]:
#Step2-1:EDA2 Missing Values - They use ?
cc_apps.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


In [17]:
#Step2-1:EDA2 Missing Values wraggling
cc_apps = cc_apps.replace('?',np.NaN)
cc_apps.isnull().sum().sort_values(ascending=False)

13    13
1     12
0     12
6      9
5      9
4      6
3      6
15     0
14     0
12     0
11     0
10     0
9      0
8      0
7      0
2      0
dtype: int64

In [18]:
#Step2-1:EDA2 Missing Values wraggling  - two part, 
#part1 categorical, part2: num 
for col in cc_apps.columns:
    # Check if the column is of object type
    if cc_apps[col].dtype == 'object':
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])
cc_apps.isnull().sum().sort_values(ascending=False)

15    0
14    0
13    0
12    0
11    0
10    0
9     0
8     0
7     0
6     0
5     0
4     0
3     0
2     0
1     0
0     0
dtype: int64

In [21]:
#Step2-1:EDA2 Missing Values wraggling  - two part, 
#part1 categorical, part2: num 

#imp = SimpleImputer(missing_values=np.nan, strategy='median')
#imp.fit(cc_apps)
#df = pd.DataFrame(data=imp.transform(cc_apps) , columns=cc_apps.columns)

cc_apps.fillna(cc_apps.mean(), inplace=True)
cc_apps.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,b,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


In [23]:
#Step3-1:Wraggling /Preprocessing 
# Part1: Object To Numeric

le=LabelEncoder()
# Iterate over all the values of each column and extract their dtypes
for col in cc_apps.columns:
    # Compare if the dtype is object
    if cc_apps[col].dtype=='object':
    # Use LabelEncoder to do the numeric transformation
        cc_apps[col]=le.fit_transform(cc_apps[col])
cc_apps.tail(17)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,1,140,2.000,3,3,5,4,2.000,0,0,0,0,0,89,17,1
674,0,214,2.500,2,1,7,4,0.210,0,0,0,0,0,90,246,1
675,0,248,1.040,2,1,0,8,0.665,0,0,0,0,0,84,237,1
676,0,153,10.665,2,1,11,4,0.085,0,1,12,1,0,40,3,1
677,1,36,7.250,2,1,10,8,0.040,0,1,1,0,0,31,1,1
678,0,20,10.210,2,1,6,3,0.000,0,0,0,0,0,0,50,1
679,0,42,1.250,2,1,2,8,0.000,0,0,0,0,0,0,0,1
680,1,37,0.290,2,1,9,8,0.290,0,0,0,0,0,96,364,1
681,1,125,1.000,3,3,4,4,3.000,0,0,0,0,0,60,537,1
682,1,12,3.290,2,1,7,8,0.335,0,0,0,1,0,44,2,1


In [24]:
#Step3-2:Wraggling /Preprocessing 
# Part2: Split
# Import train_test_split
# ... YOUR CODE FOR TASK 7 ...

# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop([11, 13], axis=1)
cc_apps = cc_apps.to_numpy()

# Segregate features and labels into separate variables
X,y = cc_apps[:,0:13] , cc_apps[:,13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)


In [27]:
#Step3-3:Wraggling /Preprocessing 
# Part3: Scale
#from sklearn.preprocessing import MinMaxScaler
# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [28]:
#Step4: build model
# Import LogisticRegression
# ... YOUR CODE FOR TASK 9 ...
from sklearn.linear_model import LogisticRegression
# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
# ... YOUR CODE FOR TASK 9 ...
logreg.fit(rescaledX_train,y_train)

LogisticRegression()

In [30]:
#Step3-1 Baseline checking
models_list = []
models_list.append(('CART', DecisionTreeClassifier()))
models_list.append(('SVM', SVC())) 
models_list.append(('NB', GaussianNB()))
models_list.append(('KNN', KNeighborsClassifier()))

In [37]:
#Step3-1 Baseline checking-Choosing Model
num_folds = 10
results = []
names = []

for name, model in models_list:
    kfold = KFold(n_splits=num_folds, random_state=123)
    start = time.time()
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    end = time.time()
    results.append(cv_results)
    names.append(name)
    print( "%s: %f (%f) (run time: %f)" % (name, cv_results.mean(), cv_results.std(), end-start))

c:\users\jadel\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
c:\users\jadel\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
c:\users\jadel\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
c:\users\jadel\appdata\local\programs\python\python38\lib\site-p

CART: 0.820305 (0.042385) (run time: 0.057957)
SVM: 0.655458 (0.089893) (run time: 0.115936)
NB: 0.798520 (0.051029) (run time: 0.026983)
KNN: 0.703377 (0.083700) (run time: 0.079954)


In [36]:
#Step3-2 Baseline checking-Choose Model- with scaler
import warnings

# Standardize the dataset
pipelines = []

pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART',
                                                                        DecisionTreeClassifier())])))
pipelines.append(('ScaledSVM', Pipeline([('Scaler', StandardScaler()),('SVM', SVC( ))])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB',
                                                                      GaussianNB())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',
                                                                       KNeighborsClassifier())])))
results = []
names = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    kfold = KFold(n_splits=num_folds, random_state=123)
    for name, model in pipelines:
        start = time.time()
        cv_results = cross_val_score(model, rescaledX_train, y_train, cv=kfold, scoring='accuracy')
        end = time.time()
        results.append(cv_results)
        names.append(name)
        print( "%s: %f (%f) (run time: %f)" % (name, cv_results.mean(), cv_results.std(), end-start))

ScaledCART: 0.831267 (0.031243) (run time: 0.083953)
ScaledSVM: 0.868039 (0.040320) (run time: 0.108943)
ScaledNB: 0.820167 (0.055397) (run time: 0.043974)
ScaledKNN: 0.846161 (0.034816) (run time: 0.108937)


In [58]:
#Step3-3 Fine tune SVM
scaler=StandardScaler().fit(X_train)
rescaledX=scaler.transform(X_train)
c_values=[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.00]
kernal_value=['linear','poly','rbf','sigmoid']
#max_iter=[100,150,200]
#tol=[0.01,0.001,0.0001]
param_grid=dict(C=c_values, kernel=kernal_value)
model=SVC()
kfold = KFold(n_splits=num_folds, random_state=21)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=kfold)
grid_result = grid.fit(rescaledX, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

c:\users\jadel\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


Best: 0.868039 using {'C': 0.9, 'kernel': 'rbf'}
0.861517 (0.025716) with: {'C': 0.1, 'kernel': 'linear'}
0.714154 (0.102792) with: {'C': 0.1, 'kernel': 'poly'}
0.863645 (0.047628) with: {'C': 0.1, 'kernel': 'rbf'}
0.863691 (0.027182) with: {'C': 0.1, 'kernel': 'sigmoid'}
0.863645 (0.023813) with: {'C': 0.3, 'kernel': 'linear'}
0.844126 (0.050402) with: {'C': 0.3, 'kernel': 'poly'}
0.863645 (0.032244) with: {'C': 0.3, 'kernel': 'rbf'}
0.863691 (0.027182) with: {'C': 0.3, 'kernel': 'sigmoid'}
0.861517 (0.023891) with: {'C': 0.5, 'kernel': 'linear'}
0.841998 (0.039981) with: {'C': 0.5, 'kernel': 'poly'}
0.863599 (0.042486) with: {'C': 0.5, 'kernel': 'rbf'}
0.865865 (0.028407) with: {'C': 0.5, 'kernel': 'sigmoid'}
0.861517 (0.023891) with: {'C': 0.7, 'kernel': 'linear'}
0.850648 (0.050769) with: {'C': 0.7, 'kernel': 'poly'}
0.865819 (0.039756) with: {'C': 0.7, 'kernel': 'rbf'}
0.861517 (0.032240) with: {'C': 0.7, 'kernel': 'sigmoid'}
0.861517 (0.023891) with: {'C': 0.9, 'kernel': 'linear'

In [84]:
#Step4 Build best model
##0.868039 using {'C': 0.9, 'kernel': 'rbf'}
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB

#from sklearn.linear_model import LogisticRegresson
# Instantiate a LogisticRegression classifier with default parameter values
#model = SVC(C=0.9,kernel='rbf') #0.85
#model = DecisionTreeClassifier() #0.86
#model = GaussianNB()
model = LogisticRegression(max_iter=100, tol=0.01)
model.fit(rescaledX_train,y_train)
y_pred = logreg.predict(rescaledX_test)
print("Accuracy of logistic regression classifier: ", model.score(rescaledX_test, y_test))
confusion_matrix(y_test,y_pred)

Accuracy of logistic regression classifier:  0.8421052631578947


array([[94,  9],
       [27, 98]], dtype=int64)

In [85]:
#Step5: Fine Tunning if using LogisticRegression

# Define the grid of values for tol and max_iter
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

param_grid = dict(tol=tol, max_iter=max_iter)
grid_model = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
rescaledX = scaler.fit_transform(X)
grid_model_result = grid_model.fit(rescaledX, y)
best_score, best_params = grid_model_result.best_score_,grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.846377 using {'max_iter': 100, 'tol': 0.01}


In [ ]:
#Step 6: Submission file generation 


t = testFile.set_index('id')
del t['Unnamed: 32']
X_test_scaled = scaler.transform(t)
pred = model.predict(X_test_scaled)
d = {'id': t.index, 'prediction': pred}
submission_df = pd.DataFrame(data=d)
submission_df

submission_df.to_csv('submissions.csv',index=False)